In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

#  Scraping Information about Berlin

In [7]:
# Loading the HTML
url = 'https://www.wikipedia.org/wiki/Berlin'
response = requests.get(url)
berlin_soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
berlin_soup.find_all(class_="infobox-data")

[<td class="infobox-data">Germany</td>,
 <td class="infobox-data"><a href="/wiki/Boroughs_and_neighborhoods_of_Berlin" title="Boroughs and neighborhoods of Berlin">Berlin</a></td>,
 <td class="infobox-data agent"><a href="/wiki/Abgeordnetenhaus_of_Berlin" title="Abgeordnetenhaus of Berlin">Abgeordnetenhaus of Berlin</a></td>,
 <td class="infobox-data"><a href="/wiki/Kai_Wegner" title="Kai Wegner">Kai Wegner</a> (<a href="/wiki/Christian_Democratic_Union_of_Germany" title="Christian Democratic Union of Germany">CDU</a>)</td>,
 <td class="infobox-data">4 (of 69)</td>,
 <td class="infobox-data"><a href="/wiki/Results_of_the_2021_German_federal_election#Berlin" title="Results of the 2021 German federal election">29 (of 736)</a></td>,
 <td class="infobox-data">891.3 km<sup>2</sup> (344.1 sq mi)</td>,
 <td class="infobox-data">3,743 km<sup>2</sup> (1,445 sq mi)</td>,
 <td class="infobox-data">30,546 km<sup>2</sup> (11,794 sq mi)</td>,
 <td class="infobox-data">34 m (112 ft)</td>,
 <td class=

In [9]:
berlin_soup.find(class_="infobox-data")

<td class="infobox-data">Germany</td>

In [10]:
# Extracting the country information
berlin_country = berlin_soup.find(class_="infobox-data").get_text()
berlin_country

'Germany'

In [11]:
# Extracting the longitude information
berlin_longitude = berlin_soup.find(class_="longitude").get_text()
berlin_longitude

'13°24′18″E'

In [12]:
# Extracting the latitude information
berlin_latitude = berlin_soup.find(class_="latitude").get_text()
berlin_latitude

'52°31′12″N'

#### Changing the longitude and latitude format to a decimal 

In [14]:
!pip install lat-lon-parser

In [15]:
from lat_lon_parser import parse

In [16]:
# Converting the latitude to a decimal 
parse(berlin_latitude)

52.519999999999996

In [17]:
# Converting the longitude to a decimal
parse(berlin_longitude)

13.405000000000001

## Scraping City Data for multiple Cities

In [18]:
cities = ["Berlin", "Hamburg", "Munich", "Cologne", "Stuttgart"]
# Creating a single list to keep all the data
city_data = []

for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # Extracting the relevant information
    country = city_soup.find(class_="infobox-data").get_text()
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()

    #  Appending a dictionary of values to the list for each city
    city_data.append({"city": city,
                     "country": country,
                     "latitude": parse(city_latitude),# latitude in decimal format
                     "longitude": parse(city_longitude) # longitude in decimal format
                    })

cities_df = pd.DataFrame(city_data)
cities_df


,city,country,latitude,longitude
0,Berlin,Germany,52.520000,13.405000
1,Hamburg,Germany,53.550000,10.000000
2,Munich,Germany,48.137500,11.575000
3,Cologne,Germany,50.936389,6.952778
4,Stuttgart,Germany,48.777500,9.180000


## Defining a Function to Automate Population Data Extraction

In [19]:
def cities_dataframe(cities):

# Creating a single list to keep all the data
city_data = []

for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # Extracting the relevant information
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()
    country = city_soup.find(class_="infobox-data").get_text()


    #  Appending a dictionary of values to the list for each city
    city_data.append({"city_name": city,
                    "country": country,
                    "latitude": parse(city_latitude), # latitude in decimal format
                    "longitude": parse(city_longitude), # longitude in decimal format
                    })

  return pd.DataFrame(city_data)


In [20]:
list_of_cities = cities = ["Berlin", "Hamburg", "Munich", "Cologne", "Stuttgart"]

cities_df = cities_dataframe(list_of_cities)
cities_df

,city_name,country,latitude,longitude
0,Berlin,Germany,52.520000,13.405000
1,Hamburg,Germany,53.550000,10.000000
2,Munich,Germany,48.137500,11.575000
3,Cologne,Germany,50.936389,6.952778
4,Stuttgart,Germany,48.777500,9.180000


Usage of Functions -> Would be easier for Gans to include cities later if they want to expand

## Scraping Population Data for Berlin

In [23]:
berlin_soup.find(string="Population").find_next()

<div class="ib-settlement-fn"><span class="nowrap"> </span>(2023-12-31)<sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[4]</a></sup></div>

In [24]:
berlin_soup.find(string="Population").find_next("td")

<td class="infobox-data">3,878,100</td>

In [25]:
berlin_population = berlin_soup.find(string="Population").find_next("td").get_text()
berlin_population

'3,878,100'

In [26]:
berlin_population_clean = int(berlin_population.replace(",", ""))
berlin_population_clean

3878100

In [27]:
# Check if the datatype of the population has actually been changed
print(type(berlin_population_clean))

<class 'int'>


## Scraping Population Data for multiple cities 

In [28]:
from datetime import datetime # to get today's date

cities = ["Berlin", "Hamburg", "Munich", "Cologne", "Stuttgart"]

# Creating a single list to keep all the data
population_data = []

for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # Extracting the relevant information
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = int(city_population.replace(",", ""))
    today = datetime.today().strftime("%d.%m.%Y")

    #  Appending a dictionary of values to the list for each city
    population_data.append({"city_name": city,
                     "population": city_population_clean,
                     "population_timestamp": today
                    })

population_data = pd.DataFrame(population_data)
population_data

,city_name,population,population_timestamp
0,Berlin,3878100,11.07.2024
1,Hamburg,1964021,11.07.2024
2,Munich,1512491,11.07.2024
3,Cologne,1084831,11.07.2024
4,Stuttgart,632865,11.07.2024


## Defining a Function to Automate Population Data Extraction

In [29]:
def populations_dataframe(cities):
# Creating a single list to keep all the data
    population_data = []

    for city in cities:
        url = f"https://www.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting the relevant information
        city_population = city_soup.find(string="Population").find_next("td").get_text()
        city_population_clean = int(city_population.replace(",", ""))
        today = datetime.today().strftime("%d.%m.%Y")

    #  Appending a dictionary of values to the list for each city
        population_data.append({"city_name": city,
                     "population": city_population_clean,
                     "population_timestamp": today
                    })

    return pd.DataFrame(population_data)

In [30]:
# Calling the populations function
cities = ["Berlin", "Hamburg", "Munich", "Cologne", "Stuttgart"]
population_df = populations_dataframe(cities)
population_df

,city_name,population,population_timestamp
0,Berlin,3878100,11.07.2024
1,Hamburg,1964021,11.07.2024
2,Munich,1512491,11.07.2024
3,Cologne,1084831,11.07.2024
4,Stuttgart,632865,11.07.2024


In [31]:
 !pip install sqlalchemy
 !pip install pymysql

In [ ]:
# This was added later to not show the password
import pw_safe

In [32]:
# Defining the connection string to the MySQL database
schema = "gans"
host = "35.240.108.232"
user = "root"
password = pw_safe.sql_root
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [33]:
# Pushing the cities information to the database
cities_df.to_sql('cities',
                  if_exists='',
                  con=connection_string,
                  index=False)

5

In [34]:
cities_df

,city_name,country,latitude,longitude
0,Berlin,Germany,52.520000,13.405000
1,Hamburg,Germany,53.550000,10.000000
2,Munich,Germany,48.137500,11.575000
3,Cologne,Germany,50.936389,6.952778
4,Stuttgart,Germany,48.777500,9.180000


In [37]:
population_data

,city_name,population,population_timestamp
0,Berlin,3878100,11.07.2024
1,Hamburg,1964021,11.07.2024
2,Munich,1512491,11.07.2024
3,Cologne,1084831,11.07.2024
4,Stuttgart,632865,11.07.2024


In [38]:
# Pulling the automatically created city_id from the database
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,city_id,city_name,country,latitude,longitude
0,1,Berlin,Germany,52.520000,13.405000
1,2,Hamburg,Germany,53.550000,10.000000
2,3,Munich,Germany,48.137500,11.575000
3,4,Cologne,Germany,50.936389,6.952778
4,5,Stuttgart,Germany,48.777500,9.180000


In [39]:
population_data_new = pd.merge(population_data, cities_from_sql[['city_id', 'city_name']], on='city_name', how='left')
population_data_new

,city_name,population,population_timestamp,city_id
0,Berlin,3878100,11.07.2024,1
1,Hamburg,1964021,11.07.2024,2
2,Munich,1512491,11.07.2024,3
3,Cologne,1084831,11.07.2024,4
4,Stuttgart,632865,11.07.2024,5


In [40]:
# Dropping redundant columns
population_data_new.drop(columns=['city_name'], inplace=True)

In [41]:
population_data_new

,population,population_timestamp,city_id
0,3878100,11.07.2024,1
1,1964021,11.07.2024,2
2,1512491,11.07.2024,3
3,1084831,11.07.2024,4
4,632865,11.07.2024,5


In [42]:
# Changing the format of the timestamp into a SQL suitable form 
population_data_new['population_timestamp'] = pd.to_datetime(population_data_new['population_timestamp'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')


In [43]:
population_data_new

,population,population_timestamp,city_id
0,3878100,2024-07-11,1
1,1964021,2024-07-11,2
2,1512491,2024-07-11,3
3,1084831,2024-07-11,4
4,632865,2024-07-11,5


In [44]:
# Pushing the population information to the database
population_data_new.to_sql('populations',
                  if_exists='replace',
                  con=connection_string,
                  index=False)

5